In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

pd.options.display.float_format = '{:.2f}'.format  #pandas 取消科學記號，只顯示小數點第二位

In [12]:
bureau=pd.read_csv("../../../../homecredit/bureau.csv") 
train = pd.read_csv("../../../../homecredit/application_train.csv")

In [6]:
bureau.shape

(1716428, 17)

In [3]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
bureau=bureau.sort_values(by=['SK_ID_CURR','SK_ID_BUREAU'])

In [9]:
bureau.isna().sum()/bureau.shape[0]

SK_ID_CURR               0.00
SK_ID_BUREAU             0.00
CREDIT_ACTIVE            0.00
CREDIT_CURRENCY          0.00
DAYS_CREDIT              0.00
CREDIT_DAY_OVERDUE       0.00
DAYS_CREDIT_ENDDATE      0.06
DAYS_ENDDATE_FACT        0.37
AMT_CREDIT_MAX_OVERDUE   0.66
CNT_CREDIT_PROLONG       0.00
AMT_CREDIT_SUM           0.00
AMT_CREDIT_SUM_DEBT      0.15
AMT_CREDIT_SUM_LIMIT     0.34
AMT_CREDIT_SUM_OVERDUE   0.00
CREDIT_TYPE              0.00
DAYS_CREDIT_UPDATE       0.00
AMT_ANNUITY              0.71
dtype: float64

In [4]:
bureau_fill = bureau.copy()

## <a id=06>6.載入home-application-train</a>

### <a id=062>提取Train_Target </a>

In [13]:
app_train_target = train[['SK_ID_CURR','TARGET','EXT_SOURCE_3']]
app_train_target.head()

,SK_ID_CURR,TARGET,EXT_SOURCE_3
0,100002,1,0.14
1,100003,0,NaN
2,100004,0,0.73
3,100006,0,NaN
4,100007,0,NaN


In [ ]:
bureau_train_target=pd.merge(bureau, app_train_target,how='left')
bureau_train_target.head()

* a1：新增5個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold、all))，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的次數。
* a2：新增4個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold)，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的比例。
* a3：新增5個欄位- CREDIT_CURRENCY_currency(1、2、3、4、all))，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的次數。
* a4：新增4個欄位- CREDIT_CURRENCY_currency(1、2、3、4)，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的比例。
* a5：新增16個欄位- CREDIT_TYPE (Another type of loan、Car loan...、all))，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的次數。
* a6：新增15個欄位- CREDIT_TYPE (Another type of loan、Car loan...)，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的比例。
    


In [ ]:
bur1 = bureau.sort_values(by=['SK_ID_CURR','SK_ID_BUREAU']).copy()


#a1：新增5個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold、all)，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的次數。

#列出同一個SK_ID_BUREAU，發生STATUS不同情況的次數
bur1_1=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_ACTIVE',aggfunc=['count'],margins=True, fill_value=0)
bur1_1.rename(columns={'Active':'CREDIT_ACTIVE_Active_count',
                                        'Bad debt':'CREDIT_ACTIVE_Bad_debt_count',
                                        'Closed':'CREDIT_ACTIVE_Closed_count',
                                        'Sold':'CREDIT_ACTIVE_Sold_count',
                                        'All':'CREDIT_ACTIVE_all_count'},inplace=True)  

#a2：新增4個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold)，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的比例。

#列出同一個SK_ID_BUREAU，發生STATUS不同情況的比例
bur1_2=pd.crosstab(bur1['SK_ID_CURR'], bur1['CREDIT_ACTIVE'], normalize=0)
bur1_2.rename(columns={'Active':'CREDIT_ACTIVE_Active_rate',
                                           'Bad debt':'CREDIT_ACTIVE_Bad_debt_rate',
                                           'Closed':'CREDIT_ACTIVE_Closed_rate',
                                           'Sold':'CREDIT_ACTIVE_Sold_rate'},inplace=True) 


#a3：新增5個欄位- CREDIT_CURRENCY_currency(1、2、3、4、all))，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的次數。
bur1_3=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_CURRENCY',aggfunc=['count'],margins=True, fill_value=0)
bur1_3.rename(columns={ 'currency 1':'CREDIT_CURRENCY_1_count',
                                             'currency 2':'CREDIT_CURRENCY_2_count',
                                             'currency 3':'CREDIT_CURRENCY_3_count',
                                             'currency 4':'CREDIT_CURRENCY_4_count',
                                             'All':'CREDIT_CURRENCY_all_count'},inplace=True) 

#a4：新增4個欄位- CREDIT_CURRENCY_currency(1、2、3、4)，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的比例。
bur1_4=pd.crosstab(bur1['SK_ID_CURR'], bur1['CREDIT_CURRENCY'], normalize=0)
bur1_4.rename(columns={ 'currency 1':'CREDIT_CURRENCY_1_rate',
                                             'currency 2':'CREDIT_CURRENCY_2_rate',
                                             'currency 3':'CREDIT_CURRENCY_3_rate',
                                             'currency 4':'CREDIT_CURRENCY_4_rate'},inplace=True) 

#a5：新增16個欄位- CREDIT_TYPE (Another type of loan、Car loan...、all))，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的次數。
bur1_5=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_TYPE',aggfunc=['count'],margins=True, fill_value=0)
bur1_5.rename(columns={ 'Another type of loan':'CREDIT_TYPE_Another_type_of_loan_count',
                                             'Car loan':'CREDIT_TYPE_Car_loan_count',
                                             'Cash loan (non-earmarked)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
                                             'Consumer credit':'CREDIT_TYPE_Consumer_credit_count',
                                             'Credit card':'CREDIT_TYPE_Credit_card_count',
                                             'Interbank credit':'CREDIT_TYPE_Interbank_credit_count',
                                             'Loan for business development':'CREDIT_TYPE_Loan_for_business_development_count',
                                             'Loan for purchase of shares (margin lending)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
                                             'Loan for the purchase of equipment':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
                                             'Loan for working capital replenishment':'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
                                             'Microloan':'CREDIT_TYPE_Microloan_count',
                                             'Mobile operator loan':'CREDIT_TYPE_Mobile_operator_loan_count',
                                             'Mortgage':'CREDIT_TYPE_Mortgage_count',
                                             'Real estate loan':'CREDIT_TYPE_Real_estate_loan_count',
                                             'Unknown type of loan':'CREDIT_TYPE_Unknown_type_of_loan_count',
                                             'All':'CREDIT_TYPE_all_count'},inplace=True)


#a6：新增15個欄位- CREDIT_TYPE (Another type of loan、Car loan...)，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的比例。
bur1_6=pd.crosstab(bur1['SK_ID_CURR'], bur1['CREDIT_TYPE'], normalize=0)
bur1_6.rename(columns={'Another type of loan':'CREDIT_TYPE_Another_type_of_loan_rate',
                                           'Car loan':'CREDIT_TYPE_Car_loan_rate',
                                           'Cash loan (non-earmarked)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_rate',
                                           'Consumer credit':'CREDIT_TYPE_Consumer_credit_rate',
                                           'Credit card':'CREDIT_TYPE_Credit_card_rate',
                                           'Interbank credit':'CREDIT_TYPE_Interbank_credit_rate',
                                           'Loan for business development':'CREDIT_TYPE_Loan_for_business_development_rate',
                                           'Loan for purchase of shares (margin lending)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_rate',
                                           'Loan for the purchase of equipment':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_rate',
                                           'Loan for working capital replenishment':'CREDIT_TYPE_Loan_for_working_capital_replenishment_rate',
                                           'Microloan':'CREDIT_TYPE_Microloan_rate',
                                           'Mobile operator loan':'CREDIT_TYPE_Mobile_operator_loan_rate',
                                           'Mortgage':'CREDIT_TYPE_Mortgage_rate',
                                           'Real estate loan':'CREDIT_TYPE_Real_estate_loan_rate',
                                           'Unknown type of loan':'CREDIT_TYPE_Unknown_type_of_loan_rate'} ,inplace=True)
                                            
                                            
bur1_6.head()


* b1、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最少的天數
* b2、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最多的天數
* b3、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差平均的天數
* b4、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期過的次數
* b5、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最多的天數
* b6、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最少的天數
* b7、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期天數的逾期平均的天數
* b8、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最多的天數
* b9、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最少的天數
* b10、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限平均的天數
* b11、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最近的天數
* b12、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最久的天數
* b13、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止平均的天數
* b14、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的最大金額的次數
* b15、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最大的金額
* b16、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最小的金額
* b17、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的逾期平均的最大金額
* b18、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過的次數。
* b19、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最多的次數
* b20、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最少的次數
* b21、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過平均的次數
* b22、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，最大的金額
* b23、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，最小的金額
* b24、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，平均的金額
* b25、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最大的金額
* b26、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最小的金額
* b27、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，平均的金額
* b28、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最大的金額
* b29、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最小的金額
* b30、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，平均的金額
* b31、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的最大金額的次數。
* b32、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最大的金額
* b33、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最小的金額
* b34、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的逾期平均的最大金額
* b35、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最近的天數
* b36、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最久的天數
* b37、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離平均的天數
* b38、AMT_ANNUITY(約定每年要償還的金額)，最大的金額
* b39、AMT_ANNUITY(約定每年要償還的金額)，最小的金額
* b40、AMT_ANNUITY(約定每年要償還的金額)，平均的金額


In [ ]:
bur2 = bureau.sort_values(by=['SK_ID_CURR' , 'SK_ID_BUREAU']).copy()

# 新增欄位
#b1、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最少的天數
bur2['DAYS_CREDIT_recent']=bur2['DAYS_CREDIT']

#b2、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最多的天數
bur2['DAYS_CREDIT_farest']=bur2['DAYS_CREDIT']

#b3、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差平均的天數
bur2['DAYS_CREDIT_mean']=bur2['DAYS_CREDIT']

#b4、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期過的次數，1為有逾期，0為沒逾期
bur2['CREDIT_DAY_OVERDUE_count']=(bur2['CREDIT_DAY_OVERDUE']>0).replace(True,1)

#b5、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最多的天數
bur2['CREDIT_DAY_OVERDUE_max']=bur2['CREDIT_DAY_OVERDUE']

#b6、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最少的天數
bur2['CREDIT_DAY_OVERDUE_min']=bur2['CREDIT_DAY_OVERDUE']

#b7、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期天數的逾期平均的天數
#CREDIT_DAY_OVERDUE_sum/CREDIT_DAY_OVERDUE_count->(b7/b4)
bur2['CREDIT_DAY_OVERDUE_sum']=bur2['CREDIT_DAY_OVERDUE']

#b8、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最多的天數
bur2['DAYS_CREDIT_ENDDATE_max']=bur2['DAYS_CREDIT_ENDDATE']

#b9、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最少的天數
bur2['DAYS_CREDIT_ENDDATE_min']=bur2['DAYS_CREDIT_ENDDATE']

#b10、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限平均的天數
bur2['DAYS_CREDIT_ENDDATE_mean']=bur2['DAYS_CREDIT_ENDDATE']

#b11、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最近的天數
bur2['DAYS_ENDDATE_FACT_max']=bur2['DAYS_ENDDATE_FACT']

#b12、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最久的天數
bur2['DAYS_ENDDATE_FACT_min']=bur2['DAYS_ENDDATE_FACT']

#b13、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止平均的天數
#如果同一個‘SK_ID_CURR' , 有'SK_ID_BUREAU'是nan，就會少除有nan的數量
bur2['DAYS_ENDDATE_FACT_mean']=bur2['DAYS_ENDDATE_FACT']

#b14、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的最大金額的次數，1為有逾期最大金額，0為沒逾期最大金額
bur2['AMT_CREDIT_MAX_OVERDUE_count']=(bur2['AMT_CREDIT_MAX_OVERDUE']>0).replace(True,1)

#b15、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最大的金額
bur2['AMT_CREDIT_MAX_OVERDUE_max']=bur2['AMT_CREDIT_MAX_OVERDUE']

#b16、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最小的金額
bur2['AMT_CREDIT_MAX_OVERDUE_min']=bur2['AMT_CREDIT_MAX_OVERDUE']

#b17、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的逾期平均的最大金額
#AMT_CREDIT_MAX_OVERDUE_sum/AMT_CREDIT_MAX_OVERDUE_count-->(b17/b14)
bur2['AMT_CREDIT_MAX_OVERDUE_sum']=bur2['AMT_CREDIT_MAX_OVERDUE']

#b18、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過的次數，1為有延長，0為沒延長
bur2['CNT_CREDIT_PROLONG_count']=(bur2['CNT_CREDIT_PROLONG']>0).replace(True,1)

#b19、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最多的次數
bur2['CNT_CREDIT_PROLONG_max']=bur2['CNT_CREDIT_PROLONG']

#b20、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最少的次數
bur2['CNT_CREDIT_PROLONG_min']=bur2['CNT_CREDIT_PROLONG']

#b21、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過平均的次數
#CNT_CREDIT_PROLONG_sum/CNT_CREDIT_PROLONG_count->(b21/b18)
bur2['CNT_CREDIT_PROLONG_sum']=bur2['CNT_CREDIT_PROLONG']

#b22、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，最大的金額
bur2['AMT_CREDIT_SUM_max']=bur2['AMT_CREDIT_SUM']

#b23、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，最小的金額
bur2['AMT_CREDIT_SUM_min']=bur2['AMT_CREDIT_SUM']

#b24、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，平均的金額
bur2['AMT_CREDIT_SUM_mean']=bur2['AMT_CREDIT_SUM']

#b25、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最大的金額
bur2['AMT_CREDIT_SUM_DEBT_max']=bur2['AMT_CREDIT_SUM_DEBT']

#b26、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最小的金額
bur2['AMT_CREDIT_SUM_DEBT_min']=bur2['AMT_CREDIT_SUM_DEBT']

#b27、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，平均的金額
bur2['AMT_CREDIT_SUM_DEBT_mean']=bur2['AMT_CREDIT_SUM_DEBT']

#b28、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最大的金額
bur2['AMT_CREDIT_SUM_LIMIT_max']=bur2['AMT_CREDIT_SUM_LIMIT']

#b29、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最小的金額
bur2['AMT_CREDIT_SUM_LIMIT_min']=bur2['AMT_CREDIT_SUM_LIMIT']

#b30、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，平均的金額
bur2['AMT_CREDIT_SUM_LIMIT_mean']=bur2['AMT_CREDIT_SUM_LIMIT']

#b31、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的最大金額的次數，1為有逾期最大金額，0為沒逾期最大金額
bur2['AMT_CREDIT_SUM_OVERDUE_count']=(bur2['AMT_CREDIT_SUM_OVERDUE']>0).replace(True,1)

#b32、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最大的金額
bur2['AMT_CREDIT_SUM_OVERDUE_max']=bur2['AMT_CREDIT_SUM_OVERDUE']

#b33、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最小的金額
bur2['AMT_CREDIT_SUM_OVERDUE_min']=bur2['AMT_CREDIT_SUM_OVERDUE']

#b34、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的逾期平均的最大金額
#AMT_CREDIT_MAX_OVERDUE_sum/AMT_CREDIT_MAX_OVERDUE_count-->(b34/b31)
bur2['AMT_CREDIT_SUM_OVERDUE_sum']=bur2['AMT_CREDIT_SUM_OVERDUE']

#b35、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最近的天數
bur2['DAYS_CREDIT_UPDATE_max']=bur2['DAYS_CREDIT_UPDATE']

#b36、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最久的天數
bur2['DAYS_CREDIT_UPDATE_min']=bur2['DAYS_CREDIT_UPDATE']

#b37、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離平均的天數
#如果同一個‘SK_ID_CURR' , 有'SK_ID_BUREAU'是nan，就會少除有nan的數量
bur2['DAYS_CREDIT_UPDATE_mean']=bur2['DAYS_CREDIT_UPDATE']

#b38、AMT_ANNUITY(約定每年要償還的金額)，最大的金額
bur2['AMT_ANNUITY_max']=bur2['AMT_ANNUITY']

#b39、AMT_ANNUITY(約定每年要償還的金額)，最小的金額
bur2['AMT_ANNUITY_min']=bur2['AMT_ANNUITY']

#b40、AMT_ANNUITY(約定每年要償還的金額)，平均的金額
bur2['AMT_ANNUITY_mean']=bur2['AMT_ANNUITY']

# 創造新的欄位 
num_aggregations = {
        'DAYS_CREDIT_recent':'max',
        'DAYS_CREDIT_farest':'min',
        'DAYS_CREDIT_mean':'mean',
        'CREDIT_DAY_OVERDUE_count':'sum',
        'CREDIT_DAY_OVERDUE_max':'max',
        'CREDIT_DAY_OVERDUE_min':'min',
        'CREDIT_DAY_OVERDUE_sum':'sum',
        'DAYS_CREDIT_ENDDATE_max':'max',
        'DAYS_CREDIT_ENDDATE_min':'min',
        'DAYS_CREDIT_ENDDATE_mean':'mean',
        'DAYS_ENDDATE_FACT_max':'max',
        'DAYS_ENDDATE_FACT_min':'min',
        'DAYS_ENDDATE_FACT_mean':'mean',
        'AMT_CREDIT_MAX_OVERDUE_count':'sum',
        'AMT_CREDIT_MAX_OVERDUE_max':'max',
        'AMT_CREDIT_MAX_OVERDUE_min':'min',
        'AMT_CREDIT_MAX_OVERDUE_sum':'sum',
        'CNT_CREDIT_PROLONG_count':'sum',
        'CNT_CREDIT_PROLONG_max':'max',
        'CNT_CREDIT_PROLONG_min':'min',
        'CNT_CREDIT_PROLONG_sum':'sum',
        'AMT_CREDIT_SUM_max':'max',
        'AMT_CREDIT_SUM_min':'min',
        'AMT_CREDIT_SUM_mean':'mean',
        'AMT_CREDIT_SUM_DEBT_max':'max',
        'AMT_CREDIT_SUM_DEBT_min':'min',
        'AMT_CREDIT_SUM_DEBT_mean':'mean',
        'AMT_CREDIT_SUM_LIMIT_max':'max',
        'AMT_CREDIT_SUM_LIMIT_min':'min',
        'AMT_CREDIT_SUM_LIMIT_mean':'mean',
        'AMT_CREDIT_SUM_OVERDUE_count':'sum',
        'AMT_CREDIT_SUM_OVERDUE_max':'max',
        'AMT_CREDIT_SUM_OVERDUE_min':'min',
        'AMT_CREDIT_SUM_OVERDUE_sum':'sum',
        'DAYS_CREDIT_UPDATE_max':'max',
        'DAYS_CREDIT_UPDATE_min':'min',
        'DAYS_CREDIT_UPDATE_mean':'mean',
        'AMT_ANNUITY_max':'max',
        'AMT_ANNUITY_min':'min',
        'AMT_ANNUITY_mean':'mean'
    }

bur2_1 = bur2.groupby(['SK_ID_CURR']).agg({**num_aggregations})

# 重新排列
#bur2_1 .reset_index(level=('SK_ID_CURR'),inplace=True)



#b7、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期平均的天數
#CREDIT_DAY_OVERDUE_sum/CREDIT_DAY_OVERDUE_count->(b7/b4)
bur2_1['CREDIT_DAY_OVERDUE_mean']=bur2_1['CREDIT_DAY_OVERDUE_sum']/bur2_1['CREDIT_DAY_OVERDUE_count']

#b17、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的最大平均的金額
#AMT_CREDIT_MAX_OVERDUE_sum/AMT_CREDIT_MAX_OVERDUE_count-->(b17/b14)
bur2_1['AMT_CREDIT_MAX_OVERDUE_mean']=bur2_1['AMT_CREDIT_MAX_OVERDUE_sum']/bur2_1['AMT_CREDIT_MAX_OVERDUE_count']

#b21、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過平均的次數
#CREDIT_DAY_OVERDUE_sum/CREDIT_DAY_OVERDUE_count->(b21/b18)
bur2_1['CNT_CREDIT_PROLONG_mean']=bur2_1['CNT_CREDIT_PROLONG_sum']/bur2_1['CNT_CREDIT_PROLONG_count']

#b34、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的逾期平均的最大金額
#AMT_CREDIT_SUM_OVERDUE_sum/AMT_CREDIT_SUM_OVERDUE_count-->(b34/b31)
bur2_1['AMT_CREDIT_SUM_OVERDUE_mean']=bur2_1['AMT_CREDIT_SUM_OVERDUE_sum']/bur2_1['AMT_CREDIT_SUM_OVERDUE_count']


bur2_1.head(20)
bur2_1.shape

In [ ]:
bur2.query('AMT_CREDIT_SUM==0').head(20)

In [ ]:
bur2_1.query('AMT_CREDIT_SUM_OVERDUE_mean>0').head()

* c1：最大的逾期金額(AMT_CREDIT_MAX_OVERDUE)是什麼貸款類別

In [ ]:

#c1：最大的逾期金額(AMT_CREDIT_MAX_OVERDUE)是什麼貸款類別

bur3 = bureau[bureau['AMT_CREDIT_MAX_OVERDUE']>0].sort_values(by=['SK_ID_CURR' , 'AMT_CREDIT_MAX_OVERDUE']).copy()

bur3['AMT_CREDIT_MAX_OVERDUE_CREDIT_TYPE'] = bur3['CREDIT_TYPE']

bur3_1 = bur3.groupby(['SK_ID_CURR']).agg({'AMT_CREDIT_MAX_OVERDUE_CREDIT_TYPE':'last'})
                                    
# # 重新排列
#bur3_1.reset_index(level=('SK_ID_CURR'),inplace=True)

bur3_1.head()
bur3_1.shape

* 合併所有新增特徵欄位
* (bur1_1,bur1_2,bur1_3,bur1_4,bur1_5,bur1_6,bur2_1,bur3_1)

In [ ]:
#合併所有新增特徵欄位
bur_all=[bur1_1,bur1_2,bur1_3,bur1_4,bur1_5,bur1_6,bur2_1,bur3_1]
bur_all=pd.concat(bur_all , axis=1)

# 重新排列
bur_all.reset_index(level=('SK_ID_CURR'),inplace=True)
bur_all.head()

* 合併所有新增特徵欄位到bureau

In [ ]:
#合併所有新增特徵欄位到bureau
bureau_FE=pd.merge(bureau,bur_all,how='left')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
bureau_FE.head(200)

In [ ]:
bureau_FE.shape

In [ ]:
#bureau_FE.to_csv("../../merge_data/bureau_FE.csv",index = False) #轉成csv，存到資策會

[[Back to Top]](#索引)

* chang有列ht沒列:

* a1：新增5個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold、all))，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的次數。
* a3：新增5個欄位- CREDIT_CURRENCY_currency(1、2、3、4、all))，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的次數。
* a5：新增16個欄位- CREDIT_TYPE (Another type of loan、Car loan...、all))，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的次數。
* b1、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最少的天數
* b2、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最多的天數
* b3、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差平均的天數
* b4、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期過的次數
* b6、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最少的天數
* b8、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最多的天數
* b9、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最少的天數
* x b10、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限平均的天數
* b11、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最近的天數
* b12、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最久的天數
* b13、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止平均的天數
* b14、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的最大金額的次數
* b16、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最小的金額
* b18、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過的次數。
* b20、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最少的次數
* x b24、AMT_CREDIT_SUM(信貸局貸款紀錄的信貸金額)，平均的金額
* b26、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最小的金額
* b28、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最大的金額
* x b29、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最小的金額
* b31、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的最大金額的次數。
* b33、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最小的金額
* b35、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最近的天數
* x b36、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最久的天數
* x b37、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離平均的天數
* x b40、AMT_ANNUITY(約定每年要償還的金額)，平均的金額
* x c1：最大的逾期金額(AMT_CREDIT_MAX_OVERDUE)是什麼貸款類別


### <a id=109012101>NEW3</a>

* chang有列ht沒列要留的:

* a1：新增5個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold、all))，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的次數。
* a3：新增5個欄位- CREDIT_CURRENCY_currency(1、2、3、4、all))，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的次數。
* a5：新增16個欄位- CREDIT_TYPE (Another type of loan、Car loan...、all))，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的次數。
* b1、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最少的天數
* b2、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最多的天數
* b3、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差平均的天數
* b4、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期過的次數
* b6、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最少的天數
* b8、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最多的天數
* b9、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最少的天數
* b11、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最近的天數
* b12、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最久的天數
* b13、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止平均的天數
* b14、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的最大金額的次數
* b16、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最小的金額
* b18、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過的次數。
* b20、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最少的次數
* b26、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最小的金額
* b28、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最大的金額
* b31、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的最大金額的次數。
* b33、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最小的金額
* b35、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最近的天數


In [11]:
bur1 = bureau.sort_values(by=['SK_ID_CURR','SK_ID_BUREAU']).copy()


#a1：新增5個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold、all)，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的次數。

#列出同一個SK_ID_BUREAU，發生STATUS不同情況的次數
bur1_1=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_ACTIVE',aggfunc=['count'], fill_value=0)
bur1_1.rename(columns={'Active':'CREDIT_ACTIVE_Active_count',
                       'Bad debt':'CREDIT_ACTIVE_Bad_debt_count',
                       'Closed':'CREDIT_ACTIVE_Closed_count',
                       'Sold':'CREDIT_ACTIVE_Sold_count'},inplace=True)  


#a3：新增5個欄位- CREDIT_CURRENCY_currency(1、2、3、4、all))，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的次數。
bur1_3=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_CURRENCY',aggfunc=['count'], fill_value=0)
bur1_3.rename(columns={ 'currency 1':'CREDIT_CURRENCY_1_count',
                        'currency 2':'CREDIT_CURRENCY_2_count',
                        'currency 3':'CREDIT_CURRENCY_3_count',
                        'currency 4':'CREDIT_CURRENCY_4_count'},inplace=True) 


#a5：新增16個欄位- CREDIT_TYPE (Another type of loan、Car loan...、all))，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的次數。
bur1_5=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_TYPE',aggfunc=['count'], fill_value=0)
bur1_5.rename(columns={ 'Another type of loan':'CREDIT_TYPE_Another_type_of_loan_count',
                        'Car loan':'CREDIT_TYPE_Car_loan_count',
                        'Cash loan (non-earmarked)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
                        'Consumer credit':'CREDIT_TYPE_Consumer_credit_count',
                        'Credit card':'CREDIT_TYPE_Credit_card_count',
                        'Interbank credit':'CREDIT_TYPE_Interbank_credit_count',
                        'Loan for business development':'CREDIT_TYPE_Loan_for_business_development_count',
                        'Loan for purchase of shares (margin lending)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
                        'Loan for the purchase of equipment':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
                        'Loan for working capital replenishment':'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
                        'Microloan':'CREDIT_TYPE_Microloan_count',
                        'Mobile operator loan':'CREDIT_TYPE_Mobile_operator_loan_count',
                        'Mortgage':'CREDIT_TYPE_Mortgage_count',
                        'Real estate loan':'CREDIT_TYPE_Real_estate_loan_count',
                        'Unknown type of loan':'CREDIT_TYPE_Unknown_type_of_loan_count'},inplace=True)



                                            
# bur1_5.head()
bur1_5.tail()

count                             \
                                      SK_ID_BUREAU                              
CREDIT_TYPE CREDIT_TYPE_Another_type_of_loan_count CREDIT_TYPE_Car_loan_count   
SK_ID_CURR                                                                      
456249                                           1                          0   
456250                                           0                          0   
456253                                           0                          0   
456254                                           0                          0   
456255                                           0                          0   

                                                         \
                                                          
CREDIT_TYPE CREDIT_TYPE_Cash_loan_(non-earmarked)_count   
SK_ID_CURR                                                
456249                                                0   
456250                                                0   
456253                                                0   
456254                                                0   
456255                                                0   

                                                                             \
                                                                              
CREDIT_TYPE CREDIT_TYPE_Consumer_credit_count CREDIT_TYPE_Credit_card_count   
SK_ID_CURR                                                                    
456249                                      9                             3   
456250                                      2                             1   
456253                                      3                             1   
456254                                      1                             0   
456255                                      9                             2   

                                                \
                                                 
CREDIT_TYPE CREDIT_TYPE_Interbank_credit_count   
SK_ID_CURR                                       
456249                                       0   
456250                                       0   
456253                                       0   
456254                                       0   
456255                                       0   

                                                             \
                                                              
CREDIT_TYPE CREDIT_TYPE_Loan_for_business_development_count   
SK_ID_CURR                                                    
456249                                                    0   
456250                                                    0   
456253                                                    0   
456254                                                    0   
456255                                                    0   

                                                                            \
                                                                             
CREDIT_TYPE CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count   
SK_ID_CURR                                                                   
456249                                                       0               
456250                                                       0               
456253                                                       0               
456254                                                       0               
456255                                                       0               

                                                                  \
                                                                   
CREDIT_TYPE CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count   
SK_ID_CURR                                                         
456249                                                       0     
456250   

In [12]:
bur2 = bureau.sort_values(by=['SK_ID_CURR' , 'SK_ID_BUREAU']).copy()

# 新增欄位
#b1、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最少的天數
bur2['DAYS_CREDIT_recent']=bur2['DAYS_CREDIT']

#b2、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差最多的天數
bur2['DAYS_CREDIT_farest']=bur2['DAYS_CREDIT']

#b3、DAYS_CREDIT(信貸局信貸與申請貸款差了幾天)，差平均的天數
bur2['DAYS_CREDIT_mean']=bur2['DAYS_CREDIT']

#b4、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，有逾期過的次數，1為有逾期，0為沒逾期
bur2['CREDIT_DAY_OVERDUE_count']=(bur2['CREDIT_DAY_OVERDUE']>0).replace(True,1)

#b6、CREDIT_DAY_OVERDUE(在申請貸款時，信貸局信貸的逾期天數)，逾期最少的天數
bur2['CREDIT_DAY_OVERDUE_min']=bur2['CREDIT_DAY_OVERDUE']

#b8、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最多的天數
bur2['DAYS_CREDIT_ENDDATE_max']=bur2['DAYS_CREDIT_ENDDATE']

#b9、DAYS_CREDIT_ENDDATE(申請“房屋信貸”時，信貸局信貸的剩餘期限)，剩餘期限最少的天數
bur2['DAYS_CREDIT_ENDDATE_min']=bur2['DAYS_CREDIT_ENDDATE']

#b11、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最近的天數
bur2['DAYS_ENDDATE_FACT_max']=bur2['DAYS_ENDDATE_FACT']

#b12、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止最久的天數
bur2['DAYS_ENDDATE_FACT_min']=bur2['DAYS_ENDDATE_FACT']

#b13、DAYS_ENDDATE_FACT(申請“房屋信貸”時，信貸局信貸終止後的天數)，距離終止平均的天數
#如果同一個‘SK_ID_CURR' , 有'SK_ID_BUREAU'是nan，就會少除有nan的數量
bur2['DAYS_ENDDATE_FACT_mean']=bur2['DAYS_ENDDATE_FACT']

#b14、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，有逾期的最大金額的次數，1為有逾期最大金額，0為沒逾期最大金額
bur2['AMT_CREDIT_MAX_OVERDUE_count']=(bur2['AMT_CREDIT_MAX_OVERDUE']>0).replace(True,1)

#b16、AMT_CREDIT_MAX_OVERDUE(逾期的最大金額)，逾期最小的金額
bur2['AMT_CREDIT_MAX_OVERDUE_min']=bur2['AMT_CREDIT_MAX_OVERDUE']

#b18、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過的次數，1為有延長，0為沒延長
bur2['CNT_CREDIT_PROLONG_count']=(bur2['CNT_CREDIT_PROLONG']>0).replace(True,1)

#b20、CNT_CREDIT_PROLONG(信貸局貸款紀錄延長了多少次)，有延長過最少的次數
bur2['CNT_CREDIT_PROLONG_min']=bur2['CNT_CREDIT_PROLONG']

#b26、AMT_CREDIT_SUM_DEBT(信貸局貸款紀錄的負債)，最小的金額
bur2['AMT_CREDIT_SUM_DEBT_min']=bur2['AMT_CREDIT_SUM_DEBT']

#b28、AMT_CREDIT_SUM_LIMIT(信貸局貸款紀錄的信用卡額度)，最大的金額
bur2['AMT_CREDIT_SUM_LIMIT_max']=bur2['AMT_CREDIT_SUM_LIMIT']

#b31、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，有逾期的最大金額的次數，1為有逾期最大金額，0為沒逾期最大金額
bur2['AMT_CREDIT_SUM_OVERDUE_count']=(bur2['AMT_CREDIT_SUM_OVERDUE']>0).replace(True,1)

#b33、AMT_CREDIT_SUM_OVERDUE(信貸局貸款紀錄的逾期金額)，逾期最小的金額
bur2['AMT_CREDIT_SUM_OVERDUE_min']=bur2['AMT_CREDIT_SUM_OVERDUE']

#b35、DAYS_CREDIT_UPDATE(離上次收到貸款訊息幾天了)，距離最近的天數
bur2['DAYS_CREDIT_UPDATE_max']=bur2['DAYS_CREDIT_UPDATE']



# 創造新的欄位 
num_aggregations = {
        'DAYS_CREDIT_recent':'max',
        'DAYS_CREDIT_farest':'min',
        'DAYS_CREDIT_mean':'mean',
        'CREDIT_DAY_OVERDUE_count':'sum',
        'CREDIT_DAY_OVERDUE_min':'min',
        'DAYS_CREDIT_ENDDATE_max':'max',
        'DAYS_CREDIT_ENDDATE_min':'min',
        'DAYS_ENDDATE_FACT_max':'max',
        'DAYS_ENDDATE_FACT_min':'min',
        'DAYS_ENDDATE_FACT_mean':'mean',
        'AMT_CREDIT_MAX_OVERDUE_count':'sum',        
        'AMT_CREDIT_MAX_OVERDUE_min':'min',
        'CNT_CREDIT_PROLONG_count':'sum',
        'CNT_CREDIT_PROLONG_min':'min',
        'AMT_CREDIT_SUM_DEBT_min':'min',
        'AMT_CREDIT_SUM_LIMIT_max':'max',
        'AMT_CREDIT_SUM_OVERDUE_count':'sum',
        'AMT_CREDIT_SUM_OVERDUE_min':'min',
        'DAYS_CREDIT_UPDATE_max':'max',

    }

bur2_1 = bur2.groupby(['SK_ID_CURR']).agg({**num_aggregations})

# 重新排列
#bur2_1 .reset_index(level=('SK_ID_CURR'),inplace=True)






bur2_1.head(20)
# bur2_1.shape

,DAYS_CREDIT_recent,DAYS_CREDIT_farest,DAYS_CREDIT_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_min,DAYS_CREDIT_ENDDATE_max,DAYS_CREDIT_ENDDATE_min,DAYS_ENDDATE_FACT_max,DAYS_ENDDATE_FACT_min,DAYS_ENDDATE_FACT_mean,AMT_CREDIT_MAX_OVERDUE_count,AMT_CREDIT_MAX_OVERDUE_min,CNT_CREDIT_PROLONG_count,CNT_CREDIT_PROLONG_min,AMT_CREDIT_SUM_DEBT_min,AMT_CREDIT_SUM_LIMIT_max,AMT_CREDIT_SUM_OVERDUE_count,AMT_CREDIT_SUM_OVERDUE_min,DAYS_CREDIT_UPDATE_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,
100001,-49,-1572,-735.00,0.00,0,1778.00,-1329.00,-544.00,-1328.00,-825.50,0.00,nan,0.00,0,0.00,0.00,0.00,0.00,-6
100002,-103,-1437,-874.00,0.00,0,780.00,-1072.00,-36.00,-1185.00,-697.50,3.00,0.00,0.00,0,0.00,31988.56,0.00,0.00,-7
100003,-606,-2586,-1400.75,0.00,0,1216.00,-2434.00,-540.00,-2131.00,-1097.33,0.00,0.00,0.00,0,0.00,810000.00,0.00,0.00,-43
100004,-408,-1326,-867.00,0.00,0,-382.00,-595.00,-382.00,-683.00,-532.50,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-382
100005,-62,-373,-190.67,0.00,0,1324.00,-128.00,-123.00,-123.00,-123.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-11
100007,-1149,-1149,-1149.00,0.00,0,-783.00,-783.00,-783.00,-783.00,-783.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-783
100008,-78,-1097,-757.33,0.00,0,471.00,-853.00,-790.00,-1028.00,-909.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-16
100009,-239,-2882,-1271.50,0.00,0,1402.00,-2152.00,-313.00,-2152.00,-1108.50,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-23
100010,-1138,-2741,-1939.50,0.00,0,689.00,-928.00,-1138.00,-1138.00,-1138.00,0.00,nan,0.00,0,0.00,0.00,0.00,0.00,-18


* 合併所有新增特徵欄位
* (bur1_1,bur1_3,bur1_5,bur2_1)

In [13]:
#合併所有新增特徵欄位
bur_all=[bur1_1,bur1_3,bur1_5,bur2_1,]
bur_all=pd.concat(bur_all , axis=1)

# 重新排列
bur_all.reset_index(level=('SK_ID_CURR'),inplace=True)
bur_all.rename(columns={ '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Active_count)':'CREDIT_ACTIVE_Active_count',
                         '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Bad_debt_count)':'CREDIT_ACTIVE_Bad_debt_count',
                         '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Closed_count)':'CREDIT_ACTIVE_Closed_count',
                         '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Sold_count)':'CREDIT_ACTIVE_Sold_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_1_count)':'CREDIT_CURRENCY_1_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_2_count)':'CREDIT_CURRENCY_2_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_3_count)':'CREDIT_CURRENCY_3_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_4_count)':'CREDIT_CURRENCY_4_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Another_type_of_loan_count)':'CREDIT_TYPE_Another_type_of_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Car_loan_count)':'CREDIT_TYPE_Car_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Cash_loan_(non-earmarked)_count)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Consumer_credit_count)':'CREDIT_TYPE_Consumer_credit_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Credit_card_count)':' CREDIT_TYPE_Credit_card_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Interbank_credit_count)':'CREDIT_TYPE_Interbank_credit_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_business_development_count)':'CREDIT_TYPE_Loan_for_business_development_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count)':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_working_capital_replenishment_count)':'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Microloan_count)':' CREDIT_TYPE_Microloan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Mobile_operator_loan_count)':'CREDIT_TYPE_Mobile_operator_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Mortgage_count)':'CREDIT_TYPE_Mortgage_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Real_estate_loan_count)':' CREDIT_TYPE_Real_estate_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Unknown_type_of_loan_count)':'CREDIT_TYPE_Unknown_type_of_loan_count',

                         
                        },inplace=True)
bur_all.head()

,SK_ID_CURR,"(count, SK_ID_BUREAU, CREDIT_ACTIVE_Active_count)","(count, SK_ID_BUREAU, CREDIT_ACTIVE_Bad_debt_count)","(count, SK_ID_BUREAU, CREDIT_ACTIVE_Closed_count)","(count, SK_ID_BUREAU, CREDIT_ACTIVE_Sold_count)","(count, SK_ID_BUREAU, CREDIT_CURRENCY_1_count)","(count, SK_ID_BUREAU, CREDIT_CURRENCY_2_count)","(count, SK_ID_BUREAU, CREDIT_CURRENCY_3_count)","(count, SK_ID_BUREAU, CREDIT_CURRENCY_4_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Another_type_of_loan_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Car_loan_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Cash_loan_(non-earmarked)_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Consumer_credit_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Credit_card_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Interbank_credit_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_business_development_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_working_capital_replenishment_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Microloan_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Mobile_operator_loan_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Mortgage_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Real_estate_loan_count)","(count, SK_ID_BUREAU, CREDIT_TYPE_Unknown_type_of_loan_count)",DAYS_CREDIT_recent,DAYS_CREDIT_farest,DAYS_CREDIT_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_min,DAYS_CREDIT_ENDDATE_max,DAYS_CREDIT_ENDDATE_min,DAYS_ENDDATE_FACT_max,DAYS_ENDDATE_FACT_min,DAYS_ENDDATE_FACT_mean,AMT_CREDIT_MAX_OVERDUE_count,AMT_CREDIT_MAX_OVERDUE_min,CNT_CREDIT_PROLONG_count,CNT_CREDIT_PROLONG_min,AMT_CREDIT_SUM_DEBT_min,AMT_CREDIT_SUM_LIMIT_max,AMT_CREDIT_SUM_OVERDUE_count,AMT_CREDIT_SUM_OVERDUE_min,DAYS_CREDIT_UPDATE_max
0,100001,3,0,4,0,7,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,-49,-1572,-735.00,0.00,0,1778.00,-1329.00,-544.00,-1328.00,-825.50,0.00,nan,0.00,0,0.00,0.00,0.00,0.00,-6
1,100002,2,0,6,0,8,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0,-103,-1437,-874.00,0.00,0,780.00,-1072.00,-36.00,-1185.00,-697.50,3.00,0.00,0.00,0,0.00,31988.56,0.00,0.00,-7
2,100003,1,0,3,0,4,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,-606,-2586,-1400.75,0.00,0,1216.00,-2434.00,-540.00,-2131.00,-1097.33,0.00,0.00,0.00,0,0.00,810000.00,0.00,0.00,-43
3,100004,0,0,2,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,-408,-1326,-867.00,0.00,0,-382.00,-595.00,-382.00,-683.00,-532.50,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-382
4,100005,2,0,1,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,-62,-373,-190.67,0.00,0,1324.00,-128.00,-123.00,-123.00,-123.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-11


In [ ]:
bur_all.tail()

In [ ]:
bur_all.shape

In [ ]:
#bur_all.to_csv("../../merge_data/bureau_FE_chang.csv",index = False) #轉成csv，存到資策會

In [14]:
#重新讀回，變更欄位名稱
bureau_FE_chang=pd.read_csv("../../merge_data/bureau_FE_chang.csv")
bureau_FE_chang.rename(columns={  "('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Active_count')":'CREDIT_ACTIVE_Active_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Bad_debt_count')":'CREDIT_ACTIVE_Bad_debt_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Closed_count')":'CREDIT_ACTIVE_Closed_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Sold_count')":'CREDIT_ACTIVE_Sold_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_1_count')":'CREDIT_CURRENCY_1_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_2_count')":'CREDIT_CURRENCY_2_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_3_count')":'CREDIT_CURRENCY_3_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_4_count')":'CREDIT_CURRENCY_4_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Another_type_of_loan_count')":'CREDIT_TYPE_Another_type_of_loan_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Car_loan_count')":'CREDIT_TYPE_Car_loan_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Cash_loan_(non-earmarked)_count')":'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Consumer_credit_count')":'CREDIT_TYPE_Consumer_credit_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Credit_card_count')":'CREDIT_TYPE_Credit_card_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Interbank_credit_count')":'CREDIT_TYPE_Interbank_credit_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Loan_for_business_development_count')":'CREDIT_TYPE_Loan_for_business_development_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count')":'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count')":'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Loan_for_working_capital_replenishment_count')":'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Microloan_count')":'CREDIT_TYPE_Microloan_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Mobile_operator_loan_count')":'CREDIT_TYPE_Mobile_operator_loan_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Mortgage_count')":'CREDIT_TYPE_Mortgage_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Real_estate_loan_count')":'CREDIT_TYPE_Real_estate_loan_count',
                         "('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Unknown_type_of_loan_count')":'CREDIT_TYPE_Unknown_type_of_loan_count'
                         
                        },inplace=True)

In [ ]:
#bureau_FE_chang.to_csv("../../merge_data/bureau_FE_chang.csv",index = False) #變更欄位名稱後。轉成csv，存到資策會

In [15]:
bureau_FE_chang.columns

Index(['SK_ID_CURR', 'CREDIT_ACTIVE_Active_count',
       'CREDIT_ACTIVE_Bad_debt_count', 'CREDIT_ACTIVE_Closed_count',
       'CREDIT_ACTIVE_Sold_count', 'CREDIT_CURRENCY_1_count',
       'CREDIT_CURRENCY_2_count', 'CREDIT_CURRENCY_3_count',
       'CREDIT_CURRENCY_4_count', 'CREDIT_TYPE_Another_type_of_loan_count',
       'CREDIT_TYPE_Car_loan_count',
       'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
       'CREDIT_TYPE_Consumer_credit_count', 'CREDIT_TYPE_Credit_card_count',
       'CREDIT_TYPE_Interbank_credit_count',
       'CREDIT_TYPE_Loan_for_business_development_count',
       'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
       'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
       'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
       'CREDIT_TYPE_Microloan_count', 'CREDIT_TYPE_Mobile_operator_loan_count',
       'CREDIT_TYPE_Mortgage_count', 'CREDIT_TYPE_Real_estate_loan_count',
       'CREDIT_TYPE_Unknown_type_of_loan_count', 'DA

In [16]:
bureau_FE_chang.head()

,SK_ID_CURR,CREDIT_ACTIVE_Active_count,CREDIT_ACTIVE_Bad_debt_count,CREDIT_ACTIVE_Closed_count,CREDIT_ACTIVE_Sold_count,CREDIT_CURRENCY_1_count,CREDIT_CURRENCY_2_count,CREDIT_CURRENCY_3_count,CREDIT_CURRENCY_4_count,CREDIT_TYPE_Another_type_of_loan_count,CREDIT_TYPE_Car_loan_count,CREDIT_TYPE_Cash_loan_(non-earmarked)_count,CREDIT_TYPE_Consumer_credit_count,CREDIT_TYPE_Credit_card_count,CREDIT_TYPE_Interbank_credit_count,CREDIT_TYPE_Loan_for_business_development_count,CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count,CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count,CREDIT_TYPE_Loan_for_working_capital_replenishment_count,CREDIT_TYPE_Microloan_count,CREDIT_TYPE_Mobile_operator_loan_count,CREDIT_TYPE_Mortgage_count,CREDIT_TYPE_Real_estate_loan_count,CREDIT_TYPE_Unknown_type_of_loan_count,DAYS_CREDIT_recent,DAYS_CREDIT_farest,DAYS_CREDIT_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_min,DAYS_CREDIT_ENDDATE_max,DAYS_CREDIT_ENDDATE_min,DAYS_ENDDATE_FACT_max,DAYS_ENDDATE_FACT_min,DAYS_ENDDATE_FACT_mean,AMT_CREDIT_MAX_OVERDUE_count,AMT_CREDIT_MAX_OVERDUE_min,CNT_CREDIT_PROLONG_count,CNT_CREDIT_PROLONG_min,AMT_CREDIT_SUM_DEBT_min,AMT_CREDIT_SUM_LIMIT_max,AMT_CREDIT_SUM_OVERDUE_count,AMT_CREDIT_SUM_OVERDUE_min,DAYS_CREDIT_UPDATE_max
0,100001,3,0,4,0,7,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,-49,-1572,-735.00,0.00,0,1778.00,-1329.00,-544.00,-1328.00,-825.50,0.00,nan,0.00,0,0.00,0.00,0.00,0.00,-6
1,100002,2,0,6,0,8,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0,-103,-1437,-874.00,0.00,0,780.00,-1072.00,-36.00,-1185.00,-697.50,3.00,0.00,0.00,0,0.00,31988.56,0.00,0.00,-7
2,100003,1,0,3,0,4,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,-606,-2586,-1400.75,0.00,0,1216.00,-2434.00,-540.00,-2131.00,-1097.33,0.00,0.00,0.00,0,0.00,810000.00,0.00,0.00,-43
3,100004,0,0,2,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,-408,-1326,-867.00,0.00,0,-382.00,-595.00,-382.00,-683.00,-532.50,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-382
4,100005,2,0,1,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,-62,-373,-190.67,0.00,0,1324.00,-128.00,-123.00,-123.00,-123.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,-11


* 合併所有新增特徵欄位到bureau

In [ ]:
#合併所有新增特徵欄位到bureau
bureau_FE=pd.merge(bureau,bur_all,how='left')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
bureau_FE.head(200)

In [ ]:
bureau_FE.shape

In [ ]:
#bureau_FE.to_csv("../../merge_data/bureau_FE_all(chang).csv",index = False) #轉成csv，存到資策會

[[Back to Top]](#索引)

In [ ]:
#未完成的另一種方法，不知如何加總同一個‘SK_ID_CURR'

# bur1 = bureau.sort_values(by=['SK_ID_CURR','SK_ID_BUREAU']).copy()


# #1:新增4個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold)，將同一個'SK_ID_CURR'彙整，列出CREDIT_ACTIVE發生的比例。
# #2:新增4個欄位- CREDIT_CURRENCY_currency(1、2、3、4)，將同一個'SK_ID_CURR'彙整，列出CREDIT_CURRENCY_currency(發生的比例。
# #3:新增15個欄位- CREDIT_TYPE (Another type of loan、Car loan...)，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的比例。

# # 將有特殊狀態的欄位新增 (onehot)
# categorical_columns = []
# for name in bur1:
#     if bur1[name].dtype=='object':
#         categorical_columns.append(name)
# bur1 = pd.get_dummies(bur1, columns = categorical_columns)

# bur1.rename(columns={'CREDIT_ACTIVE_Active':'CREDIT_ACTIVE_Active_rate',
#                      'CREDIT_ACTIVE_Bad debt':'CREDIT_ACTIVE_Bad_debt_rate',
#                      'CREDIT_ACTIVE_Closed':'CREDIT_ACTIVE_Closed_rate',
#                      'CREDIT_ACTIVE_Sold':'CREDIT_ACTIVE_Sold_rate',
#                      'CREDIT_CURRENCY_currency 1':'CREDIT_CURRENCY_currency_1_rate',
#                      'CREDIT_CURRENCY_currency 2':'CREDIT_CURRENCY_currency_2_rate',
#                      'CREDIT_CURRENCY_currency 3':'CREDIT_CURRENCY_currency_3_rate',
#                      'CREDIT_CURRENCY_currency 4':'CREDIT_CURRENCY_currency_4_rate',
#                      'CREDIT_TYPE_Another type of loan':'CREDIT_TYPE_Another_type_of_loan_rate',
#                      'CREDIT_TYPE_Car loan':'CREDIT_TYPE_Car_loan_rate',
#                      'CREDIT_TYPE_Cash loan (non-earmarked)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_rate',
#                      'CREDIT_TYPE_Consumer credit':'CREDIT_TYPE_Consumer_credit_rate',
#                      'CREDIT_TYPE_Credit card':'CREDIT_TYPE_Credit_card_rate',
#                      'CREDIT_TYPE_Interbank credit':'CREDIT_TYPE_Interbank_credit_rate',
#                      'CREDIT_TYPE_Loan for business development':'CREDIT_TYPE_Loan_for_business_development_rate',
#                      'CREDIT_TYPE_Loan for purchase of shares (margin lending)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_rate',
#                      'CREDIT_TYPE_Loan for the purchase of equipment':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_rate',
#                      'CREDIT_TYPE_Loan for working capital replenishment':'CREDIT_TYPE_Loan_for_working_capital_replenishment_rate',
#                      'CREDIT_TYPE_Microloan':'CREDIT_TYPE_Microloan_rate',
#                      'CREDIT_TYPE_Mobile operator loan':'CREDIT_TYPE_Mobile_operator_loan_rate',
#                      'CREDIT_TYPE_Mortgage':'CREDIT_TYPE_Mortgage_rate',
#                      'CREDIT_TYPE_Real estate loan':'CREDIT_TYPE_Real_estate_loan_rate',
#                      'CREDIT_TYPE_Unknown type of loan':'CREDIT_TYPE_Unknown_type_of_loan_rate'}
                    
#                     ,inplace=True)


# bur1['Total_SK_ID_BUREAU_count']=bur1['DAYS_CREDIT'] #SK_ID_CURR'合計


# #創造新的欄位 
# num_aggregations = {
#         'Total_SK_ID_BUREAU_count':'count',
#         'CREDIT_ACTIVE_Active_rate':'sum',
#         'CREDIT_ACTIVE_Bad_debt_rate':'sum',
#         'CREDIT_ACTIVE_Closed_rate':'sum',
#         'CREDIT_ACTIVE_Sold_rate':'sum',
#         'CREDIT_CURRENCY_currency_1_rate':'sum',
#         'CREDIT_CURRENCY_currency_2_rate':'sum',
#         'CREDIT_CURRENCY_currency_3_rate':'sum',
#         'CREDIT_CURRENCY_currency_4_rate':'sum',
#         'CREDIT_TYPE_Another_type_of_loan_rate':'sum',
#         'CREDIT_TYPE_Car_loan_rate':'sum',
#         'CREDIT_TYPE_Cash_loan_(non-earmarked)_rate':'sum',
#         'CREDIT_TYPE_Consumer_credit_rate':'sum',
#         'CREDIT_TYPE_Credit_card_rate':'sum',
#         'CREDIT_TYPE_Interbank_credit_rate':'sum',
#         'CREDIT_TYPE_Loan_for_business_development_rate':'sum',
#         'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_rate':'sum',
#         'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_rate':'sum',
#         'CREDIT_TYPE_Loan_for_working_capital_replenishment_rate':'sum',
#         'CREDIT_TYPE_Microloan_rate':'sum',
#         'CREDIT_TYPE_Mobile_operator_loan_rate':'sum',
#         'CREDIT_TYPE_Mortgage_rate':'sum',
#         'CREDIT_TYPE_Real_estate_loan_rate':'sum',
#         'CREDIT_TYPE_Unknown_type_of_loan_rate':'sum',
    
    
    
    
         

    
    
#     }


# bur_1 = bur1.groupby(['SK_ID_CURR']).agg({**num_aggregations})





# # # 重新排列
# # bur_1.reset_index(level=('SK_ID_CURR','SK_ID_BUREAU'),inplace=True)



# #a1:新增4個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold)，將同一個'SK_ID_CURR'彙整，列出CREDIT_ACTIVE發生的比例。
# #狀態發生期數除以同'SK_ID_CURR'總數

# bur_1['CREDIT_ACTIVE_Active_rate'] = bur_1['CREDIT_ACTIVE_Active_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_ACTIVE_Bad_debt_rate'] = bur_1['CREDIT_ACTIVE_Bad_debt_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_ACTIVE_Closed_rate'] = bur_1['CREDIT_ACTIVE_Closed_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_ACTIVE_Sold_rate'] = bur_1['CREDIT_ACTIVE_Sold_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_CURRENCY_currency_1_rate'] = bur_1['CREDIT_CURRENCY_currency_1_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_CURRENCY_currency_2_rate'] = bur_1['CREDIT_CURRENCY_currency_2_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_CURRENCY_currency_3_rate'] = bur_1['CREDIT_CURRENCY_currency_3_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_CURRENCY_currency 4_rate'] = bur_1['CREDIT_CURRENCY_currency_4_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Another_type_of_loan_rate'] = bur_1['CREDIT_TYPE_Another_type_of_loan_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Car_loan_rate'] =bur_1['CREDIT_TYPE_Car_loan_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Cash_loan_(non-earmarked)_rate'] = bur_1['CREDIT_TYPE_Cash_loan_(non-earmarked)_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Consumer_credit_rate'] = bur_1['CREDIT_TYPE_Consumer_credit_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Credit_card_rate'] =bur_1['CREDIT_TYPE_Credit_card_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Interbank_credit_rate'] =bur_1['CREDIT_TYPE_Interbank_credit_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Loan_for_business_development_rate'] = bur_1['CREDIT_TYPE_Loan_for_business_development_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_rate'] = bur_1['CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Loan_for_the_purchase_of_equipment_rate'] = bur_1['CREDIT_TYPE_Loan_for_the_purchase_of_equipment_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Loan_for_working_capital_replenishment_rate'] = bur_1['CREDIT_TYPE_Loan_for_working_capital_replenishment_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Microloan_rate'] = bur_1['CREDIT_TYPE_Microloan_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Mobile_operator_loan_rate'] =bur_1['CREDIT_TYPE_Mobile_operator_loan_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Mortgage_rate'] = bur_1['CREDIT_TYPE_Mortgage_rate'] /  bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Real_estate_loan_rate'] = bur_1['CREDIT_TYPE_Real_estate_loan_rate'] / bur_1['Total_SK_ID_BUREAU_count']
# bur_1['CREDIT_TYPE_Unknown_type_of_loan_rate'] = bur_1['CREDIT_TYPE_Unknown_type_of_loan_rate'] / bur_1['Total_SK_ID_BUREAU_count']



# bur1.head()

* [新增4個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold)，將同一個'SK_ID_CURR'彙整，列出CREDIT_ACTIVE發生的比例。併到bureau。](#a01)
* [新增15個欄位- CREDIT_TYPE (Another type of loan、Car loan...)，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的情況one hot encoding。併到bureau。](#a02)

In [ ]:
#4筆['CREDIT_ACTIVE']=='Bad debt')，有違約。bureau_balance查不到資料
# bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Bad debt')&(bureau_train_target['TARGET']>0),
#             ('SK_ID_CURR','SK_ID_BUREAU','TARGET','AMT_CREDIT_SUM_OVERDUE','CNT_CREDIT_PROLONG')].shape[0]
bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Bad debt')&(bureau_train_target['TARGET']>0),
            ('SK_ID_CURR','SK_ID_BUREAU','TARGET','CREDIT_ACTIVE','AMT_CREDIT_SUM_OVERDUE','CNT_CREDIT_PROLONG')].head(10)

#1筆['CREDIT_ACTIVE']=='Bad debt')&(bureau_train_target['TARGET'].isnull()。bureau_balance，stat有多筆是5
#stat==5(means DPD 120+ or sold or written off)
# bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Bad debt')&(bureau_train_target['TARGET'].isnull()),
#             ('SK_ID_CURR','SK_ID_BUREAU','TARGET','AMT_CREDIT_SUM_OVERDUE','CNT_CREDIT_PROLONG')].head(10)
#16筆bureau_train_target['CREDIT_ACTIVE']=='Bad debt')&(bureau_train_target['TARGET']==0)
#bureau_balance有資料的：5499851(41個月STATUS都是x)，
# bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Bad debt')&(bureau_train_target['TARGET']==0),
#             ('SK_ID_CURR','SK_ID_BUREAU','TARGET','CREDIT_ACTIVE','AMT_CREDIT_SUM_OVERDUE','CNT_CREDIT_PROLONG')].head(10)

In [ ]:
#共6527筆bureau_train_target['CREDIT_ACTIVE']=='Sold'
bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')].shape[0]
#874筆['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET'].isnull())
#bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET'].isnull())].shape[0]
#5082筆['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==0)
#bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==0)].shape[0]
#571筆['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)，0筆同時逾期天數、延長了多少次>0
#571筆['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)，562筆逾期天數==0,9筆逾期天數>0，延長了多少次==0
#bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)&(bureau_train_target['CREDIT_DAY_OVERDUE']>0)&(bureau_train_target['CNT_CREDIT_PROLONG']==0)].shape[0]
# bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)&(bureau_train_target['CREDIT_DAY_OVERDUE']>0)&(bureau_train_target['CNT_CREDIT_PROLONG']==0)].head()
#6筆'CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)，延長了多少次>0
#565筆'CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)，延長了多少次>0
#bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)&(bureau_train_target['CNT_CREDIT_PROLONG']==0)].shape[0]
#bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold')&(bureau_train_target['TARGET']==1)].tail(20)

In [ ]:
#由CREDIT_ACTIVE看出，違約比率Bad debt(0.2)>Sold(0.1)>Active(0.092)>Closed(0.06)
#bureau_train_target.groupby('CREDIT_ACTIVE')[['TARGET']].sum().head()
bureau_train_target.groupby('CREDIT_ACTIVE')[['TARGET']].mean().head()

#### <a id=a01>新增4個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold)，將同一個'SK_ID_CURR'彙整，列出CREDIT_ACTIVE發生的比例。併到bureau。</a>

In [ ]:
#1090114將同一個'SK_ID_CURR'彙整，列出CREDIT_ACTIVE發生的比例。併到train。
CREDIT_ACTIVE_count=pd.crosstab(bureau_train_target['SK_ID_CURR'], bureau_train_target['CREDIT_ACTIVE'],normalize=0)
CREDIT_ACTIVE_count.rename(columns={'Active':'CREDIT_ACTIVE_Active',
                                    'Bad debt':'CREDIT_ACTIVE_Bad debt',
                                    'Closed':'CREDIT_ACTIVE_Closed',
                                    'Sold':'CREDIT_ACTIVE_Sold'},inplace=True)
#CREDIT_ACTIVE_count.head()
CREDIT_ACTIVE_count=pd.merge(bureau, CREDIT_ACTIVE_count,left_on='SK_ID_CURR',right_index=True,how='left')
CREDIT_ACTIVE_count.head()

In [ ]:
#1090114將同一個'SK_ID_CURR'彙整，列出CREDIT_ACTIVE發生的次數。以便觀察target
#bureau_train_target.pivot_table('TARGET',index='SK_ID_CURR',columns='CREDIT_ACTIVE').head()
bureau_train_target.pivot_table(values=['TARGET'],index='SK_ID_CURR',columns='CREDIT_ACTIVE',aggfunc=['count'],margins=True, fill_value=0).tail(15)

#### <a id=a02> 新增15個欄位- CREDIT_TYPE (Another type of loan、Car loan...)，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的情況one hot encoding。併到bureau。</a>

* 無採金額的比例，因為有些人的金額(AMT_CREDIT_SUM)是空值，或是0

In [ ]:
#one hot encoding
CREDIT_CREDIT_TYPE_count_onehot = pd.get_dummies(bureau_train_target['CREDIT_TYPE'])
CREDIT_CREDIT_TYPE_count_onehot.head()
CREDIT_CREDIT_TYPE_count_onehot=pd.merge(bureau, CREDIT_CREDIT_TYPE_count_onehot,left_on='SK_ID_CURR',right_index=True,how='left')
CREDIT_CREDIT_TYPE_count_onehot.head(10)

* 新增15個欄位- CREDIT_TYPE (Another type of loan、Car loan...)，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的比例。併到bureau。
* 待確是否要使用


In [ ]:
#將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的比例。以便觀察target、便於匯入train
CREDIT_CREDIT_TYPE_count=pd.crosstab(bureau_train_target['SK_ID_CURR'], bureau_train_target['CREDIT_TYPE'],normalize=0)
#CREDIT_CREDIT_TYPE_count.head()
CREDIT_CREDIT_TYPE_count.tail()
CREDIT_CREDIT_TYPE_count=pd.merge(bureau, CREDIT_CREDIT_TYPE_count,left_on='SK_ID_CURR',right_index=True,how='left')
CREDIT_CREDIT_TYPE_count.head()

In [ ]:
#Loan for the purchase of equipmen最高，#4筆違約#13筆無違約
bureau_train_target.groupby('CREDIT_TYPE')[['TARGET']].mean().sort_values(by=["TARGET"])

In [ ]:
bureau[bureau['AMT_CREDIT_SUM']==0].head()

In [ ]:
#bureau_train_target.groupby(['SK_ID_CURR','AMT_CREDIT_SUM'])['CREDIT_TYPE'].agg('sum').head()
#bureau_train_target.pivot_table(values=['AMT_CREDIT_SUM'],index='SK_ID_CURR',columns='CREDIT_TYPE',aggfunc=['count'],margins=True, fill_value=0).tail(15)

In [ ]:
#共19筆CREDIT_TYPE：Loan for the purchase of equipment
# bureau_train_target[bureau_train_target["CREDIT_TYPE"]=='Loan for the purchase of equipment'].sort_values(by=["SK_ID_CURR"]).shape[0]
#4筆違約
#13筆無違約
#2筆無資料
bureau_train_target.loc[(bureau_train_target['CREDIT_TYPE']=='Loan for the purchase of equipment')&(bureau_train_target['TARGET'].isnull())].shape[0]

In [ ]:
bureau_train_target.groupby('CREDIT_TYPE')[['TARGET']].sum().sort_values(by=["TARGET"])

In [ ]:
#2種方法一樣
#bureau_train_target.groupby(['CREDIT_ACTIVE','CREDIT_TYPE'])['TARGET'].aggregate('mean').unstack()
#privot_table(aggfunc='mean'(預設)，可改sum.mean.count.min.max)
bureau_train_target.pivot_table('TARGET',index='CREDIT_ACTIVE',columns='CREDIT_TYPE')

In [ ]:
#['CREDIT_ACTIVE']=='Sold') 
#571 TARGET']>0
#5082 ['TARGET']==0
#874['TARGET'].isnull()
bureau_train_target.loc[(bureau_train_target['CREDIT_ACTIVE']=='Sold') & (bureau_train_target['TARGET'].isnull())].shape[0]

In [ ]:
6480057
bureau_train_target.loc[(bureau_train_target['SK_ID_BUREAU']==6842759)]

In [ ]:
pd.set_option("display.max_rows",None)
bureau_train_target.loc[bureau_train_target["SK_ID_CURR"]==163815].sort_values(by=["SK_ID_BUREAU"])

In [ ]:
#將DAYS_CREDIT_UPDATE切割、CREDIT_ACTIVE、CREDIT_TYPE看違約比率
# DAYS_CREDIT_UPDATE_check=pd.cut(bureau_train_target['DAYS_CREDIT_UPDATE'],[-42000,-41800,-2920,-2555,-2190,-1825,-1460,-1095,-730,-365,-1,0])
# bureau_train_target.pivot_table('TARGET',['CREDIT_ACTIVE',DAYS_CREDIT_UPDATE_check],'CREDIT_TYPE')
# DAYS_CREDIT_UPDATE_check=pd.cut(bureau_train_target['DAYS_CREDIT_UPDATE'],[-42000,-41800,-2920,-1825,-1095,-730,-365,-1,0])
# bureau_train_target.pivot_table('TARGET',['CREDIT_ACTIVE',DAYS_CREDIT_UPDATE_check],'CREDIT_TYPE')

DAYS_CREDIT_UPDATE_check=pd.cut(bureau_train_target['DAYS_CREDIT_UPDATE'],[-42000,-41800,-2920,-1825,-365,-1,0])
bureau_train_target.pivot_table('TARGET',['CREDIT_ACTIVE',DAYS_CREDIT_UPDATE_check],'CREDIT_TYPE')                                


[[Back to Top]](#索引)

### <a id=082>與測試資料驗證</a>

In [ ]:
#bureau.loc[(bureau['AMT_CREDIT_SUM_OVERDUE']>0)&(bureau['CNT_CREDIT_PROLONG']>0)].shape[0]
bureau_test_target.loc[(bureau_test_target['AMT_CREDIT_SUM_OVERDUE']>0)&(bureau_test_target['CNT_CREDIT_PROLONG']>0)&(bureau_test_target['test']==1),
            ('SK_ID_CURR','SK_ID_BUREAU','TARGET','AMT_CREDIT_SUM_OVERDUE','CNT_CREDIT_PROLONG')].shape[0]

In [ ]:
bureau_test_target.loc[(bureau_test_target['CREDIT_ACTIVE']=='Sold') & (bureau_test_target['test']>0)].shape[0]

[[Back to Top]](#索引)

## <a id=09>9.載入home-bureau_balance</a>

### <a id=091>資料的型態、大小</a>

In [ ]:
#group_3
#bureau_balance=pd.read_csv("../../datasets/homecdt_eda/bureau_balance.csv")
#資策會
#bureau_balance=pd.read_csv("../../bureau_balance.csv") # 所有資料
#家裡
bureau_balance=pd.read_csv("home-credit-default-risk/bureau_balance.csv")# 所有資料

In [ ]:
bureau_balance.head(5)

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
bureau_balanceu=bureau_balance.sort_values(by=['SK_ID_BUREAU','MONTHS_BALANCE'])
bureau_balance.head(200)

In [ ]:
total_info(bureau_balance)

[[Back to Top]](#索引)

### <a id=092>特徵篩選</a>

### <a id=109011602>NEW2</a>

* 載入bureau_balance合併的特徵欄位

In [ ]:
#資策會
#bureau_balance_FE=pd.read_csv("../../merge_data/bureau_balance_FE.csv" )

#### <a id=bb>將bureau_balance資料合併到bureau</a>

In [ ]:
#將bureau_balance資料合併到bureau特徵
bureau_balance_FE_to_bureau=pd.merge(bureau_FE, bureau_balance_FE,how='outer')

#將bureau_balance資料合併到bureau
#bureau_balance_FE_to_bureau=pd.merge(bureau, bureau_balance_FE,how='outer')


#bureau_balance_FE_to_bureau=pd.merge(bureau, bureau_balance_FE,left_on='SK_ID_BUREAU',right_index=True,how='outer')
bureau_balance_FE_to_bureau.shape
bureau_balance_FE_to_bureau.head()

In [ ]:
bureau_balance_FE_to_bureau[bureau_balance_FE_to_bureau["recent_3_status_0%"]==0].head()

In [ ]:
bureau_balance_FE_to_bureau.query('recent_3_status_0%==0').head()

In [ ]:
#bureau_balance_FE_to_bureau.to_csv('../../merge_data/bureau_balance_FE_to_bureau.csv',index = False)

[[Back to Top]](#索引)

### <a id=093>合併與訓練資料驗證</a>

In [ ]:
bureau_train_target_STATUS_count=pd.merge(bureau_train_target, STATUS_count,left_on='SK_ID_BUREAU',right_index=True,how='left')
#1716428筆
#bureau_train_target_STATUS_count.shape[0]
bureau_train_target_STATUS_count.head(100)

#### STATUS有缺942074筆資料(約54.89%）。bureau_balance資料較少

In [ ]:
# STATUS有缺942074筆資料(約54.89%）。bureau_balance資料較少
total_info(bureau_train_target_STATUS_count)

In [ ]:
#433筆TARGET']==1，bureau_train_target_STATUS_count['5']
#bureau_train_target_STATUS_count.loc[(bureau_train_target_STATUS_count['TARGET']==1)&(bureau_train_target_STATUS_count['5']>0)].shape[0]
bureau_train_target_STATUS_count.loc[(bureau_train_target_STATUS_count['TARGET']==1)&(bureau_train_target_STATUS_count['5']>0)].shape[0]

In [ ]:
#檢視同樣SK_ID_CURR發生bureau_balance的STATUS的比例，已準備合併到train
bureau_train_target_STATUS_count.groupby('SK_ID_CURR')[['TARGET','0','1','2','3','4','5','C','X']].mean().head(20)

In [ ]:
bureau_train_target_STATUS_count[bureau_train_target_STATUS_count["SK_ID_CURR"]==100001].sort_values(by=["SK_ID_BUREAU"])

In [ ]:
#???待確
#bureau_train_target_STATUS_count.pivot_table(values=['C'],index='SK_ID_CURR',columns='TARGET',aggfunc=['count'],margins=True, fill_value=0).head(5)

In [ ]:
#???待確
#bureau_train_target_STATUS_count.pivot_table(values=['C','1'],index='SK_ID_BUREAU',columns='TARGET',aggfunc=['count'],margins=True, fill_value=0).head(5)

In [ ]:
heatmap(bureau_train_target_STATUS_count)

[[Back to Top]](#索引)

## <a id=96>96.sample</a>

[[Back to Top]](#索引)

## <a id=97>97.test</a>

[[Back to Top]](#索引)

## <a id=98>98.待確</a>

In [ ]:
bureau.fillna(method='ffill',axis=1)
bureau.head(5)


#sort_day_credit_flag_fillna=sort_day_credit_flag[['SK_ID_CURR','SK_ID_BUREAU','AMT_CREDIT_SUM','DAYS_CREDIT_ENDDATE','day_credit_flag']]
#sort_day_credit_flag_fillna.head(n=15)

In [ ]:
sort_missing_data=bureau[['SK_ID_CURR','SK_ID_BUREAU','AMT_CREDIT_SUM','DAYS_CREDIT_ENDDATE']].sort_values(by='AMT_CREDIT_SUM')
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(sort_values_REGION_INCOME)
sort_missing_data.head(n=15)
#ssort_missing_data.tail(n=15)
#sort_missing_data

[[Back to Top]](#索引)

# 下面先略過

In [ ]:
bureau.apply(lambda x:x.unique().shape[0],axis=0)

In [ ]:
#bureau.columns[bureau.dtypes == 'object']
for i in bureau.columns[bureau.dtypes == 'object']:
    print(i)

In [ ]:
np.unique(bureau['CREDIT_ACTIVE'])

In [ ]:
np.unique(bureau['CREDIT_CURRENCY'])

In [ ]:
np.unique(bureau['CREDIT_TYPE'])

In [ ]:
b= pd.unique(bureau['CREDIT_ACTIVE'])
c= pd.DataFrame(b,columns=['CREDIT_ACTIVE'])
#print('CREDIT_ACTIVE', b)
c

In [ ]:
#pd.unique?

In [ ]:
d = np.unique(bureau[bureau.columns[bureau.dtypes == 'object']])
d

In [ ]:
e= pd.DataFrame(columns=[bureau.columns[bureau.dtypes == 'object']])
e

In [ ]:
#pd.concat

In [ ]:
#plt.figure(figsize=(20,20),dpi=200)
#sns.heatmap(np.abs(bureau.corr()),cmap="Blues",
#           vmin=0,
#          vmax=1,
#          square=True,
#           annot=True)
#plt.show()

In [ ]:
#plt.figure(figsize=(20,20),dpi=200)

##sns.pairplot(data=bureau, vars=["SK_ID_BUREAU", "MONTHS_BALANCE", "STATUS"])
#sns.pairplot(data=bureau, vars=bureau.columns[0:5])
#plt.show()

In [ ]:
#plt.figure(figsize=(20,20),dpi=200)

##sns.pairplot(data=bureau, vars=["SK_ID_BUREAU", "MONTHS_BALANCE", "STATUS"])
#sns.pairplot(data=bureau, vars=bureau.columns[0:10])
#plt.show()

In [ ]:
#plt.figure(figsize=(20,20),dpi=200)

##sns.pairplot(data=bureau, vars=["SK_ID_BUREAU", "MONTHS_BALANCE", "STATUS"])
#sns.pairplot(data=bureau, vars=bureau.columns[10:18])
#plt.show()

In [ ]:
#plt.figure(figsize=(20,12),dpi=200)

#sns.pairplot(data=bureau, vars=["SK_ID_CURR", "SK_ID_BUREAU", "DAYS_CREDIT"])
#sns.pairplot(data=bureau, vars=bureau.columns)
#plt.show()

[[Back to Top]](#索引)

## <a id=99>99.印出所有報告</a>

--pandas_profiling.ProfileReport()--印出所有報告

In [ ]:
df = bureau.copy()
df.head(15)

In [ ]:
df2 = df.iloc[0:10]
df2

In [ ]:
total_info(df2)

In [ ]:
pfr = pandas_profiling.ProfileReport(df2)
pfr

In [ ]:
pfr.to_file('test2-report.html')

[[Back to Top]](#索引)